## **bronze-getting-historical-data**
<p>This notebook extracts raw historical stock price data—such as open, high, low, close, and volume—for multiple tickers from Yahoo Finance. It serves as the base time-series dataset for price trend, volatility, or return analysis in the silver and gold layers.</p>

#### **Importing necessary libraries**

In [1]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, 343d6851-00aa-4fd1-b70f-177c55ed6fed, 5, Finished, Available, Finished)

#### **Variables**

In [2]:
file_name = "yfinance_data"
layer_name = "bronze"

StatementMeta(, 343d6851-00aa-4fd1-b70f-177c55ed6fed, 6, Finished, Available, Finished)

#### **Public API**

The following are the publicly available classes and functions exposed by the yfinance package:

- **Ticker**: Class for accessing single ticker data.
- **Tickers**: Class for handling multiple tickers.
- **Market**: Class for accessing market summary.
- **download**: Function to download market data for multiple tickers.
- **Search**: Class for accessing search results.
- **Lookup**: Class for looking up tickers.
- **WebSocket**: Class for synchronously streaming live market data.
- **AsyncWebSocket**: Class for asynchronously streaming live market data.
- **Sector**: Domain class for accessing sector information.
- **Industry**: Domain class for accessing industry information.
- **Market**: Class for accessing market status & summary.
- **EquityQuery**: Class to build equity query filters.
- **FundQuery**: Class to build fund query filters.
- **screen**: Run equity/fund queries.
- **enable_debug_mode**: Function to enable debug mode for logging.
- **set_tz_cache_location**: Function to set the timezone cache location.


#### **Step 1: Load the company codes (tickers) from the CSV**

In [3]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, 343d6851-00aa-4fd1-b70f-177c55ed6fed, 7, Finished, Available, Finished)

#### **Step 2: Fetch stock history for each ticker**

In [4]:
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        hist = tickers.history(period="max")

        if not hist.empty:
            hist.reset_index(inplace=True)

            hist['Company'] = i

            output.append(hist)
        else:
            print(f"No data found for {i}")
    except Exception as e:
        print(f"An error occurred for {i}: {e}")


StatementMeta(, 343d6851-00aa-4fd1-b70f-177c55ed6fed, 8, Finished, Available, Finished)

No data found for ETRN
No data found for ALTR
No data found for SWN
No data found for NYCB
No data found for BERY
No data found for EDR
No data found for ALTM
No data found for MRO


#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [5]:
if output:
    final_df = pd.concat(output, ignore_index=True)
    spark_df = spark.createDataFrame(final_df)

    spark_df.write.mode("overwrite").partitionBy('Date').parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, 343d6851-00aa-4fd1-b70f-177c55ed6fed, 9, Finished, Available, Finished)

[SUCCESS] Stock data written to Lakehouse
